In [105]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
# from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import load_model
from keras.utils import np_utils


In [106]:
print(tf.__version__)

2.4.1


In [107]:
gpus=tf.config.list_physical_devices()
print("gpus available", gpus)
# tf.config.experimental.set_memory_growth(gpus[0], True)
# print(tf.test.gpu_device_name())

gpus available [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### read dataset X and Y

In [193]:
subject_id=2
base_path = "features_notwelch/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features_notwelch/data_chb{:02d}/features_{}_00.npy'.format(subject_id,subject_id))
y=load('features_notwelch/data_chb{:02d}/targets_{}_00.npy'.format(subject_id,subject_id))
for subject_id in (2,5,9,11,12):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X=np.concatenate((X, load('features_notwelch/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features_notwelch/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(files)
    print(X.shape)


72
(12968, 208)
78
(27189, 208)
38
(51560, 208)
70
(64000, 208)
42
(71760, 208)


In [194]:
X_shape, y_shape = X.shape, y.shape
print(X_shape, y_shape)

(71760, 208) (71760,)


In [195]:
def generate_column_names():
    channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
    col_names=[]
    for ch in channel_order:
        # print(ch)
        for i in range(0,9):
            col_names.append(str(ch)+'-'+str(i))
    # col_names.append('rms')
    # col_names.append('sum')
    col_names.append('patient')
    # print(col_names.shape)
    return col_names


In [196]:
df=pd.DataFrame(data=X, columns=generate_column_names())
df['target']=y
print(df.shape)
df=df.dropna()
print(df.shape)
df.reset_index(drop=True, inplace=True)

(71760, 209)
(71759, 209)


In [197]:
# df['seizure_number']=0
# seizure_number=1
# for i in range(1, len(df)):
#     # if df.iloc[i]['target']==2 and df.iloc[i-1]['target']==2:
#     #     df.iloc[i]['seizure_number']=df.iloc[i-1]['seizure_number']
#     if (df.iloc[i]['target']==2):
#         df.at[i,'seizure_number']=seizure_number
#     if (df.iloc[i]['target']!=2 and df.iloc[i-1]['target']==2 ):
#         seizure_number+=1
# test_df=pd.DataFrame()
# for patient in pd.unique(df['patient']):
#     # patdf=df[df[207]==patient]
#     last_seizure=max(df[df['patient']==patient]['seizure_number'])
#     test_df = test_df.append(df[(df['patient']==patient) & (df['seizure_number']==last_seizure)], ignore_index=False)
# test_df
# # df.to_csv(r'df2.csv')


In [220]:
df['seizure_number']=0

seizure_number=1
for i in range(1, len(df)):
    # if df.iloc[i]['target']==2 and df.iloc[i-1]['target']==2:
    #     df.iloc[i]['seizure_number']=df.iloc[i-1]['seizure_number']
    if df.iloc[i]['target']==2:
        df.at[i,'seizure_number']=seizure_number
    if (df.iloc[i]['target']!=2 and df.iloc[i-1]['target']==2 ):
        seizure_number+=1
df
# df.to_csv(r'df.csv')



,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
0,0.018578,0.095241,0.026204,0.026413,0.267436,0.122472,0.402273,0.072887,1553.302246,0.014406,...,0.020839,0.067815,0.342813,0.082369,0.219504,0.057476,2179.300537,2.0,0,0
1,0.017195,0.116917,0.028268,0.020735,0.287858,0.113165,0.369562,0.061889,2126.991699,0.011974,...,0.008063,0.016239,0.368241,0.079414,0.265690,0.065642,5608.150391,2.0,0,0
2,0.023806,0.160434,0.033086,0.027919,0.314111,0.112796,0.305782,0.049759,1298.781006,0.011973,...,0.004080,0.009234,0.416091,0.060620,0.254375,0.063313,12217.144531,2.0,0,0
3,0.024970,0.143778,0.034336,0.036952,0.303799,0.135452,0.329680,0.059968,754.909790,0.007608,...,0.002542,0.005959,0.413690,0.065602,0.252079,0.058523,14063.621094,2.0,0,0
4,0.002849,0.048261,0.002897,0.003678,0.153665,0.130988,0.510873,0.140414,12543.096680,0.002975,...,0.001261,0.004485,0.265586,0.088062,0.393971,0.085730,18503.945312,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71754,0.014313,0.153195,0.006152,0.014744,0.287621,0.075952,0.289818,0.106538,7432.621582,0.042097,...,0.002883,0.010063,0.412930,0.066247,0.244435,0.086542,2857.702148,12.0,3,0
71755,0.013912,0.131002,0.005290,0.012155,0.273308,0.073352,0.323711,0.118133,8756.443359,0.036799,...,0.003953,0.008558,0.271229,0.079665,0.265976,0.107630,1545.392578,12.0,3,0
71756,0.022538,0.186055,0.008216,0.021497,0.289533,0.059521,0.311911,0.096910,4677.033691,0.043397,...,0.010345,0.012555,0.241365,0.096627,0.233801,0.089307,952.947632,12.0,3,0
71757,0.047645,0.311022,0.020933,0.061205,0.291337,0.052105,0.178151,0.059963,1808.921387,0.048682,...,0.019911,0.028009,0.315189,0.075089,0.202171,0.086369,561.303345,12.0,3,0


In [221]:
test_df=pd.DataFrame()
for patient in pd.unique(df['patient']):
    # patdf=df[df[207]==patient]
    print(patient)
    last_seizure=max(df[df['patient']==patient]['seizure_number'])
    test_df = test_df.append(df[(df['patient']==patient) & (df['seizure_number']==last_seizure)], ignore_index=False)
test_df

2.0
5.0
9.0
11.0
12.0


,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
7162,0.024722,0.167940,0.022232,0.040775,0.372914,0.077367,0.253885,0.053518,1946.139648,0.023040,...,0.005336,0.014324,0.345691,0.072008,0.255091,0.056759,3028.504395,2.0,2,3
7163,0.011673,0.140104,0.005736,0.013079,0.467539,0.068067,0.237187,0.047278,4959.291992,0.029791,...,0.007012,0.022235,0.356219,0.065431,0.226064,0.044866,1992.042847,2.0,2,3
7164,0.012789,0.161469,0.016808,0.020162,0.463838,0.060573,0.216059,0.040374,4813.878418,0.015255,...,0.004499,0.014204,0.294616,0.076901,0.262006,0.047901,2765.090088,2.0,2,3
7165,0.026937,0.173863,0.068735,0.064965,0.363154,0.056674,0.210230,0.039481,2711.432373,0.013499,...,0.004150,0.010867,0.279594,0.081538,0.272143,0.058216,4110.312988,2.0,2,3
7166,0.021215,0.137705,0.071709,0.057182,0.398063,0.055010,0.224288,0.041962,3424.862793,0.015650,...,0.007664,0.013128,0.336973,0.069166,0.258830,0.061734,5443.549805,2.0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71573,0.057434,0.449707,0.033225,0.051070,0.433269,0.009048,0.022220,0.004940,413.782379,0.058471,...,0.160693,0.272260,0.042583,0.000433,0.001810,0.001268,273.556274,12.0,2,39
71574,0.055667,0.435668,0.037922,0.050661,0.435502,0.009454,0.022743,0.005223,379.738708,0.058253,...,0.141379,0.150678,0.057322,0.000523,0.001846,0.001473,226.755234,12.0,2,39
71575,0.062079,0.444765,0.043678,0.049211,0.426007,0.008424,0.021301,0.005031,392.124817,0.057531,...,0.161199,0.189323,0.056296,0.000672,0.003524,0.001930,206.779114,12.0,2,39
71576,0.053513,0.503538,0.030657,0.035249,0.403018,0.006758,0.017817,0.003927,497.231781,0.049436,...,0.139491,0.192732,0.052969,0.000643,0.003340,0.001971,221.903259,12.0,2,39


In [222]:
test_df[test_df['target']!=2]

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number


In [223]:
# test_df.to_csv(r'test_df.csv')


In [224]:
df = df[df.index.isin(test_df.index) == False]

### train test split

In [225]:
df_interictal=df[df['target']==0]
# print(df_interictal.shape)
df_interictal=df_interictal.sample(frac=0.2)
df_preictal=df[df['target']==2]
df_preictal = df_preictal.replace({'target': 2}, 1)
test_df = test_df.replace({'target': 2}, 1)
interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
print(df_interictal.shape, df_preictal.shape)

# df_interictal=df[df['target']==0]
# df_interictal=df_interictal.sample(frac=0.2)
# df_preictal=df[df['target']==2]
# interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
# interictal_shape= df_interictal.shape

# print(df_interictal.shape,  df_preictal.shape)


(12494, 210) (1519, 210)


In [226]:
# X_interictal=np.array(df_interictal[df.columns[0:-2]]).astype('float32')
# y_interictal=np.array(df_interictal['target']).astype('float32')
# X_preictal_train  =np.array(df_preictal[df.columns[:-2]]).astype('float32')
# y_preictal_train  =np.array(df_preictal['target']).astype('float32')
X_interictal=np.array(df_interictal[df.columns[:-2]]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal_train  =np.array(df_preictal[df.columns[:-2]]).astype('float32')
y_preictal_train  =np.array(df_preictal['target']).astype('float32')

In [227]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal_train.shape)
print(y_preictal_train.shape)

(12494, 208)
(12494,)
(1519, 208)
(1519,)


In [228]:
df_interictal[df.columns[:-2]]

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-0,T8-P8-1-1,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient
45972,0.004331,0.074158,0.002722,0.003501,0.175945,0.080066,0.442735,0.154198,65254.500000,0.002624,...,0.013827,0.201816,0.012313,0.028431,0.263969,0.073814,0.294482,0.090222,9290.288086,9.0
25143,0.097475,0.264733,0.150336,0.146393,0.297588,0.032300,0.095158,0.012931,1384.454834,0.100922,...,0.170164,0.247275,0.200316,0.229634,0.177482,0.030912,0.076588,0.009490,507.993683,5.0
29553,0.013507,0.106932,0.012695,0.010930,0.186941,0.079720,0.394058,0.159222,21730.957031,0.007660,...,0.029254,0.160777,0.002360,0.012516,0.254266,0.068270,0.344016,0.108444,19499.537109,9.0
56595,0.002107,0.118466,0.002050,0.002152,0.471416,0.078310,0.265949,0.040193,35021.382812,0.016120,...,0.044210,0.165352,0.010857,0.036968,0.293260,0.067112,0.296906,0.082849,1210.145264,11.0
23658,0.013228,0.139170,0.079380,0.058290,0.227415,0.065873,0.297846,0.101728,7768.779297,0.011053,...,0.127749,0.153463,0.293808,0.368119,0.094259,0.013920,0.080695,0.020532,1282.029785,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19865,0.036160,0.180962,0.123352,0.093771,0.329097,0.042889,0.187185,0.053526,2206.135498,0.013610,...,0.012585,0.197628,0.003468,0.004686,0.374663,0.058613,0.246230,0.074285,46744.343750,5.0
2350,0.014446,0.141418,0.032889,0.025948,0.258065,0.077724,0.333375,0.105910,3483.398438,0.005409,...,0.015250,0.209534,0.003944,0.010755,0.385972,0.073166,0.233872,0.053010,9562.818359,2.0
5618,0.010031,0.080618,0.020130,0.019936,0.220296,0.071255,0.379176,0.155025,9990.822266,0.001578,...,0.001123,0.094524,0.000090,0.000300,0.494216,0.066682,0.261224,0.055521,483009.375000,2.0
70609,0.080106,0.305004,0.104403,0.096742,0.289549,0.029230,0.100593,0.037282,482.822998,0.038154,...,0.088845,0.477305,0.234229,0.247230,0.149067,0.001708,0.027211,0.009689,2295.086182,12.0


### train test split

In [229]:
from sklearn.model_selection import train_test_split


In [230]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
# X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [231]:
interictal_test_df=pd.DataFrame(data=X_interictal_test, columns=generate_column_names())
interictal_test_df['target']=y_interictal_test
interictal_test_df
# test_df=test_df.append(interictal_test_df, ignore_index=True)
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
7162,0.024722,0.167940,0.022232,0.040775,0.372914,0.077367,0.253885,0.053518,1946.139648,0.023040,...,0.005336,0.014324,0.345691,0.072008,0.255091,0.056759,3028.504395,2.0,1,3
7163,0.011673,0.140104,0.005736,0.013079,0.467539,0.068067,0.237187,0.047278,4959.291992,0.029791,...,0.007012,0.022235,0.356219,0.065431,0.226064,0.044866,1992.042847,2.0,1,3
7164,0.012789,0.161469,0.016808,0.020162,0.463838,0.060573,0.216059,0.040374,4813.878418,0.015255,...,0.004499,0.014204,0.294616,0.076901,0.262006,0.047901,2765.090088,2.0,1,3
7165,0.026937,0.173863,0.068735,0.064965,0.363154,0.056674,0.210230,0.039481,2711.432373,0.013499,...,0.004150,0.010867,0.279594,0.081538,0.272143,0.058216,4110.312988,2.0,1,3
7166,0.021215,0.137705,0.071709,0.057182,0.398063,0.055010,0.224288,0.041962,3424.862793,0.015650,...,0.007664,0.013128,0.336973,0.069166,0.258830,0.061734,5443.549805,2.0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71573,0.057434,0.449707,0.033225,0.051070,0.433269,0.009048,0.022220,0.004940,413.782379,0.058471,...,0.160693,0.272260,0.042583,0.000433,0.001810,0.001268,273.556274,12.0,1,39
71574,0.055667,0.435668,0.037922,0.050661,0.435502,0.009454,0.022743,0.005223,379.738708,0.058253,...,0.141379,0.150678,0.057322,0.000523,0.001846,0.001473,226.755234,12.0,1,39
71575,0.062079,0.444765,0.043678,0.049211,0.426007,0.008424,0.021301,0.005031,392.124817,0.057531,...,0.161199,0.189323,0.056296,0.000672,0.003524,0.001930,206.779114,12.0,1,39
71576,0.053513,0.503538,0.030657,0.035249,0.403018,0.006758,0.017817,0.003927,497.231781,0.049436,...,0.139491,0.192732,0.052969,0.000643,0.003340,0.001971,221.903259,12.0,1,39


In [232]:
# X_train = np.concatenate((X_interictal_train, X_preictal_train))
# y_train = np.concatenate((y_interictal_train, y_preictal_train))
# X_test = np.array(test_df[df.columns[:-2]]).astype('float32')
# y_test  =np.array(test_df['target']).astype('float32')
X_train = np.concatenate((X_interictal_train, X_preictal_train))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
X_test = np.array(test_df[df.columns[:-2]]).astype('float32')
y_test  =np.array(test_df['target']).astype('float32')
X_test = np.concatenate((X_test, X_interictal_test))
y_test  =np.concatenate((y_test, y_interictal_test))

In [233]:
X_train_shape =X_train.shape
X_test_shape = X_test.shape
y_train_shape =y_train.shape 
y_test_shape = y_test.shape
print(X_train_shape)
print(X_test_shape)
print(y_train_shape)
print(y_test_shape)

(12763, 208)
(1531, 208)
(12763,)
(1531,)


### Normalization

In [234]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

### encoding

In [235]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
# encoded_Y = encoder.fit_transform(y)
enc_y_train = np_utils.to_categorical(y_train)
enc_y_test = np_utils.to_categorical(y_test)
print(enc_y_test.shape)
print(enc_y_test.max())

(1531, 2)
1.0


### training the model

In [236]:
from datetime import datetime

In [237]:
model = keras.Sequential(
    [
        Dense(256, name="layer1", input_shape=(208,), activation = 'linear' ),
        Dense(256, name="hidden1", activation = 'linear'),
        Dense(128, name="hidden2", activation = 'linear'),
        Dense(128, name="hidden3", activation = 'linear'),
        Dense(64, name="hidden4", activation = 'linear'),
        Dense(32, name="hidden5", activation = 'linear'),
        Dense(2, name="output", activation = 'sigmoid'),
    ]
)
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 256)               53504     
_________________________________________________________________
hidden1 (Dense)              (None, 256)               65792     
_________________________________________________________________
hidden2 (Dense)              (None, 128)               32896     
_________________________________________________________________
hidden3 (Dense)              (None, 128)               16512     
_________________________________________________________________
hidden4 (Dense)              (None, 64)                8256      
_________________________________________________________________
hidden5 (Dense)              (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 2)                

In [238]:
opt=keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)
])

In [239]:
before = datetime.now()
before_time =before.strftime("%H:%M:%S")
print(before_time)
class_weight = {0: 0.1, 1: 0.9}
model.fit(X_train, enc_y_train, epochs=200, batch_size= 32, validation_data=(X_test, enc_y_test), class_weight=class_weight)

after = datetime.now()
after_time =after.strftime("%H:%M:%S")
print(after_time)

print ("saving new model")
model.save("DNNseizure.h5")

racy: 0.8672 - false_negatives_9: 875.7650 - val_loss: 0.4991 - val_accuracy: 0.7747 - val_false_negatives_9: 340.0000
Epoch 106/200
399/399 [==============================] - 1s 2ms/step - loss: 0.0527 - accuracy: 0.8707 - false_negatives_9: 847.3825 - val_loss: 0.5008 - val_accuracy: 0.7662 - val_false_negatives_9: 346.0000
Epoch 107/200
399/399 [==============================] - 1s 2ms/step - loss: 0.0545 - accuracy: 0.8594 - false_negatives_9: 896.7625 - val_loss: 0.5016 - val_accuracy: 0.7890 - val_false_negatives_9: 322.0000
Epoch 108/200
399/399 [==============================] - 1s 2ms/step - loss: 0.0519 - accuracy: 0.8769 - false_negatives_9: 829.8650 - val_loss: 0.4803 - val_accuracy: 0.8021 - val_false_negatives_9: 302.0000
Epoch 109/200
399/399 [==============================] - 1s 2ms/step - loss: 0.0586 - accuracy: 0.8556 - false_negatives_9: 910.4400 - val_loss: 0.5885 - val_accuracy: 0.7675 - val_false_negatives_9: 355.0000
Epoch 110/200
399/399 [======================

In [240]:
model.evaluate(X_test, enc_y_test)

48/48 [==============================] - 0s 1ms/step - loss: 0.5150 - accuracy: 0.7609 - false_negatives_9: 366.0000


[0.5149813890457153, 0.7609405517578125, 366.0]

In [241]:
# loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
# y_pred=loaded_clf.predict(X_test)
y_pred1=model.predict(X_test)
y_pred1

array([[9.9389040e-01, 6.1482787e-03],
       [9.7671568e-01, 2.3385584e-02],
       [9.6795124e-01, 3.2190770e-02],
       ...,
       [7.3068601e-01, 2.6935992e-01],
       [9.9997461e-01, 2.5775556e-05],
       [9.6060961e-01, 3.9581358e-02]], dtype=float32)

In [242]:
y_pred=[0]*len(y_pred1)
i=0
for pr in y_pred1:
    y_pred[i]=np.argmax(pr)
    i+=1
y_pred=np.array(y_pred)
y_pred.shape



(1531,)

In [243]:
# loaded_clf.get_params(deep=True)

In [244]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:1015, FP:235, FN:131, TP:150


In [245]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

specificity= 0.812 , sensitivity= 0.5338078291814946


In [246]:
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

# print("Training Accuracy: %d"%(trainAcc*100)+"%")
# print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

X size =(71760, 208), y size = (71760,)
interictal size =(12494, 210), preictal size = (1519, 210)
train size =(12763, 208), test size = (1531, 208)
TN:1015, FP:235, FN:131, TP:150
specificity= 0.812 , sensitivity= 0.5338078291814946


### grid search

In [247]:
# from numpy import mean
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RepeatedStratifiedKFold

In [248]:
# balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:50}, {0:1,2:100}]
# costs=[1.0,10.0,100.0]
# param_grid = dict(C=costs, class_weight=balance)

In [249]:
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
# grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_weighted')


In [250]:
# grid_result = grid.fit(X_train, y_train)

In [251]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))